In [1]:
# Import Dependencies
from pymongo import MongoClient
from config import cloudM, cloudMpassword
import pandas as pd
import numpy as np
import datetime as dt
import csv
import json
import re

In [2]:
#create a variable for columns to keep
#did not include the note columns because it would drop some data due to null
columns = ["Country", 
            "Economy: Real GDP (purchasing power parity)", 
            "Economy: Real GDP growth rate",
            "Economy: Real GDP per capita",
            "Economy: Current account balance",
            "Economy: Exports",
            "Economy: Exports - partners",
            "Economy: Reserves of foreign exchange and gold",
            "Energy: Electricity access - electrification - total population",
            "Economy: Credit ratings - Moody's rating",
            "Economy: Credit ratings - Standard & Poors rating",
            "Economy: Credit ratings - Fitch rating",
            "Economy: Unemployment rate",
            "Economy: Population below poverty line",
            "Economy: Household income or consumption by percentage share - highest 10%",
            "People and Society: Population",
            "People and Society: Nationality - noun",
            "People and Society: Nationality - adjective",
            "People and Society: Languages",
            "People and Society: Religions",
            "People and Society: Age structure - 0-14 years",
            "People and Society: Age structure - 15-24 years",
            "People and Society: Age structure - 25-54 years",
            "People and Society: Age structure - 55-64 years",
            "People and Society: Median age - total",
            "People and Society: Median age - male",
            "People and Society: Median age - female",
            "People and Society: Population growth rate",
            "People and Society: Birth rate",
            "People and Society: Death rate",
            "People and Society: Net migration rate",
            "People and Society: Urbanization - urban population",
            "People and Society: Sex ratio - at birth",
            "People and Society: Sex ratio - 65 years and over",
            "People and Society: Sex ratio - total population",
            "People and Society: Infant mortality rate - total",
            "People and Society: Life expectancy at birth - total population",
            "People and Society: Life expectancy at birth - male",
            "People and Society: Life expectancy at birth - female",
            "People and Society: Total fertility rate",
            "People and Society: Obesity - adult prevalence rate",
            "People and Society: Tobacco use - total",
            "Communications: Internet users - total",
            "Communications: Internet users - percent of population",
            "Military and Security: Military expenditures",
            "Military and Security: Military service age and obligation",
            "Terrorism: Terrorist group(s)", "Military and Security: Military - note"]                                

In [3]:
#load data and create df to clean
data = pd.read_csv("countries.csv")
#previouly determined all fields were objects, load as string for cleaning
df = pd.DataFrame(data).astype("string")
df = df.loc[:, columns].copy()
df.head()

,Country,Economy: Real GDP (purchasing power parity),Economy: Real GDP growth rate,Economy: Real GDP per capita,Economy: Current account balance,Economy: Exports,Economy: Exports - partners,Economy: Reserves of foreign exchange and gold,Energy: Electricity access - electrification - total population,Economy: Credit ratings - Moody's rating,...,People and Society: Life expectancy at birth - female,People and Society: Total fertility rate,People and Society: Obesity - adult prevalence rate,People and Society: Tobacco use - total,Communications: Internet users - total,Communications: Internet users - percent of population,Military and Security: Military expenditures,Military and Security: Military service age and obligation,Terrorism: Terrorist group(s),Military and Security: Military - note
0,Afghanistan,$77.04 billion (2020 est.)$78.56 billion (2019...,2.7% (2017 est.)2.2% (2016 est.)1% (2015 est.),"$2,000 (2020 est.)$2,100 (2019 est.)$2,000 (20...",$1.014 billion (2017 est.)$1.409 billion (2016...,$1.48 billion (2020 est.),"United Arab Emirates 45%, Pakistan 24%, India ...",$7.187 billion (31 December 2017 est.)$6.901 b...,99% (2018),<NA>,...,55.28 years (2022 est.),4.62 children born/woman (2022 est.),5.5% (2016),23.3% (2020 est.),"7,007,101 (2020 est.)",18% (2020 est.),3.3% of GDP (2019) (approximately $2.35 billio...,not available,Haqqani Taliban Network; Harakat ul-Mujahidin;...,"as of early 2022, the Taliban’s primary securi..."
1,Akrotiri,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,defense is the responsibility of the UK; Akrot...
2,Albania,$37.73 billion (2020 est.)$39.02 billion (2019...,2.24% (2019 est.)4.07% (2018 est.)3.8% (2017 e...,"$13,300 (2020 est.)$13,700 (2019 est.)$13,300 ...",-$908 million (2017 est.)-$899 million (2016 e...,$3.47 billion (2020 est.) note: data are in cu...,"Italy 45%, Spain 8%, Germany 6%, Greece 5%, Fr...",$3.59 billion (31 December 2017 est.)$3.109 bi...,100% (2020),B1 (2007),...,82.33 years (2022 est.),1.54 children born/woman (2022 est.),21.7% (2016),22.4% (2020 est.),"2,043,251 (2020 est.)",72% (2020 est.),1.4% of GDP (2021 est.)1.3% of GDP (2020)1.5% ...,19 is the legal minimum age for voluntary mili...,Islamic Revolutionary Guard Corps/Qods Force; ...,Albania officially became a member of NATO in ...
3,Algeria,$468.4 billion (2020 est.)$495.56 billion (201...,1.4% (2017 est.)3.2% (2016 est.)3.7% (2015 est.),"$10,700 (2020 est.)$11,500 (2019 est.)$11,600 ...",-$22.1 billion (2017 est.)-$26.47 billion (201...,$38.32 billion (2019 est.) note: data are in c...,"Italy 13%, France 13%, Spain 12%, United State...",$97.89 billion (31 December 2017 est.)$114.7 b...,99.4% (2019),<NA>,...,79.57 years (2022 est.),2.51 children born/woman (2022 est.),27.4% (2016),21% (2020 est.),"27,626,157 (2020 est.)",63% (2020 est.),5.6% of GDP (2021 est.)6.7% of GDP (2020 est.)...,18 is the legal minimum age for voluntary mili...,al-Qa'ida in the Islamic Maghreb (AQIM); Islam...,the ANP has played a large role in the country...
4,American Samoa,$658 million (2016 est.)$674.9 million (2015 e...,-2.5% (2016 est.)1.2% (2015 est.)1% (2014 est.),"$11,200 (2016 est.)$11,300 (2015 est.)$11,200 ...",<NA>,$428 million (2016 est.)$427 million (2015 est.),"Australia 25%, Ghana 19%, Indonesia 15.6%, Bur...",<NA>,<NA>,<NA>,...,77.97 years (2022 est.),2.21 children born/woman (2022 est.),<NA>,<NA>,"17,147 (2020 est.)",31% (2020 est.),<NA>,<NA>,<NA>,defense is the responsibility of the US


In [4]:
#understand df composition
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 48 columns):
 #   Column                                                                      Non-Null Count  Dtype 
---  ------                                                                      --------------  ----- 
 0   Country                                                                     258 non-null    string
 1   Economy: Real GDP (purchasing power parity)                                 229 non-null    string
 2   Economy: Real GDP growth rate                                               224 non-null    string
 3   Economy: Real GDP per capita                                                229 non-null    string
 4   Economy: Current account balance                                            205 non-null    string
 5   Economy: Exports                                                            225 non-null    string
 6   Economy: Exports - partners                               

In [5]:
#find null vallues
for column in df.columns:
    print(f'column {column} has {df[column].isnull().sum()} null values')

column Country has 0 null values
column Economy: Real GDP (purchasing power parity) has 29 null values
column Economy: Real GDP growth rate has 34 null values
column Economy: Real GDP per capita has 29 null values
column Economy: Current account balance has 53 null values
column Economy: Exports has 33 null values
column Economy: Exports - partners has 33 null values
column Economy: Reserves of foreign exchange and gold has 64 null values
column Energy: Electricity access - electrification - total population has 37 null values
column Economy: Credit ratings - Moody's rating has 118 null values
column Economy: Credit ratings - Standard & Poors rating has 124 null values
column Economy: Credit ratings - Fitch rating has 139 null values
column Economy: Unemployment rate has 39 null values
column Economy: Population below poverty line has 78 null values
column Economy: Household income or consumption by percentage share - highest 10% has 106 null values
column People and Society: Populatio

In [6]:
#drop null values & determing number of rows
df = df.dropna()
df.count()

Country                                                                       49
Economy: Real GDP (purchasing power parity)                                   49
Economy: Real GDP growth rate                                                 49
Economy: Real GDP per capita                                                  49
Economy: Current account balance                                              49
Economy: Exports                                                              49
Economy: Exports - partners                                                   49
Economy: Reserves of foreign exchange and gold                                49
Energy: Electricity access - electrification - total population               49
Economy: Credit ratings - Moody's rating                                      49
Economy: Credit ratings - Standard & Poors rating                             49
Economy: Credit ratings - Fitch rating                                        49
Economy: Unemployment rate  

In [7]:
#create a function to clean the economy columns
#$77.04 billion (2020 est.)$78.56 billion (2019 est.)$75.6 billion (2018 est.)
#change column name to "Real GDP (purchasing power parity) in billions"
#one column for 2020, 2019, 2018
#remove billion from column
#remove (2020 est.) remove $

In [8]:
#create a for loop to remove billions from Real GDP (purchasing power parity)
df_c= df.rename(columns = {"Economy: Real GDP (purchasing power parity)" : "Real GDP"})
df_c.head()

,Country,Real GDP,Economy: Real GDP growth rate,Economy: Real GDP per capita,Economy: Current account balance,Economy: Exports,Economy: Exports - partners,Economy: Reserves of foreign exchange and gold,Energy: Electricity access - electrification - total population,Economy: Credit ratings - Moody's rating,...,People and Society: Life expectancy at birth - female,People and Society: Total fertility rate,People and Society: Obesity - adult prevalence rate,People and Society: Tobacco use - total,Communications: Internet users - total,Communications: Internet users - percent of population,Military and Security: Military expenditures,Military and Security: Military service age and obligation,Terrorism: Terrorist group(s),Military and Security: Military - note
10,Argentina,$893.31 billion (2020 est.)$991.52 billion (20...,-2.03% (2019 est.)-2.53% (2018 est.)2.83% (201...,"$19,700 (2020 est.)$22,100 (2019 est.)$22,800 ...",-$3.997 billion (2019 est.)-$27.049 billion (2...,$64.18 billion (2020 est.) note: data are in c...,"Brazil 16%, China 11%, United States 7%, Chile...",$55.33 billion (31 December 2017 est.)$38.43 b...,99% (2020),Ca (2020),...,81.59 years (2022 est.),2.18 children born/woman (2022 est.),28.3% (2016),24.5% (2020 est.),"39,024,016 (2020 est.)",86% (2020 est.),0.8% of GDP (2021 est.)0.8% of GDP (2020)0.7% ...,18-24 years of age for voluntary military serv...,Hizballah,Argentina has Major Non-NATO Ally (MNNA) statu...
15,Austria,$463.12 billion (2020 est.)$495.8 billion (201...,1.42% (2019 est.)2.58% (2018 est.)2.4% (2017 e...,"$51,900 (2020 est.)$55,800 (2019 est.)$55,300 ...",$12.667 billion (2019 est.)$5.989 billion (201...,$226.79 billion (2020 est.) note: data are in ...,"Germany 28%, United States 7%, Italy 6%, Switz...",$21.57 billion (31 December 2017 est.)$23.36 b...,100% (2020),Aa1 (2016),...,85.04 years (2022 est.),1.51 children born/woman (2022 est.),20.1% (2016),26.4% (2020 est.),"7,846,840 (2020 est.)",88% (2020 est.),0.9% of GDP (2021 est.)0.7% of GDP (2020 est.)...,"registration requirement at age 17, the legal ...",Islamic State of Iraq and ash-Sham (ISIS),"Austria is constitutionally non-aligned, but i..."
16,Azerbaijan,$138.51 billion (2020 est.)$144.74 billion (20...,0.1% (2017 est.)-3.1% (2016 est.)0.6% (2015 est.),"$13,700 (2020 est.)$14,400 (2019 est.)$14,200 ...",$1.685 billion (2017 est.)-$1.363 billion (201...,$15.21 billion (2020 est.) note: data are in c...,"Italy 28%, Turkey 15%, Israel 7%, Germany 5%, ...",$6.681 billion (31 December 2017 est.)$7.142 b...,100% (2020),Ba2 (2017),...,77.41 years (2022 est.),1.86 children born/woman (2022 est.),19.9% (2016),24% (2020 est.),"8,579,153 (2020 est.)",85% (2020 est.),5.2% of GDP (2021 est.)5.4% of GDP (2020 est.)...,men age 18-35 years for compulsory military se...,Islamic State of Iraq and ash-Sham (ISIS); Isl...,"since November 2020, Russia has deployed about..."
20,Bangladesh,$793.49 billion (2020 est.)$775.08 billion (20...,7.4% (2017 est.)7.2% (2016 est.)6.8% (2015 est.),"$4,800 (2020 est.)$4,800 (2019 est.)$4,400 (20...",-$5.322 billion (2017 est.)$1.391 billion (201...,$38.78 billion (2020 est.) note: data are in c...,"United States 15%, Germany 14%, United Kingdom...",$33.42 billion (31 December 2017 est.)$32.28 b...,83% (2019),Ba3 (2012),...,76.96 years (2022 est.),2.09 children born/woman (2022 est.),3.6% (2016),34.7% (2020 est.),"41,172,346 (2020 est.)",25% (2020 est.),1.2% of GDP (2021 est.)1.3% of GDP (2020 est.)...,16-21 years of age for voluntary military serv...,Harakat ul-Jihad-i-Islami/Bangladesh; Islamic ...,the military’s chief areas of focus are border...
23,Belgium,$557.11 billion (2020 est.)$594.47 billion (20...,1.41% (2019 est.)1.49% (2018 est.)1.9% (2017 e...,"$48,200 (2020 est.)$51,700 (2019 est.)$51,100 ...",$1.843 billion (2019 est.)-$4.135 billion (201...,$414.79 billion (2020 est.) note: data are in ...,"Germany 17%, France 14%, Netherlands 13%, Unit...",$26.16 billion (31 December 2017 est.)$24.1 bi...,100% (2020),A

In [9]:
real_gdp = df_c["Real GDP"]

In [10]:
real_gdp = real_gdp.str.replace("billion", " ")
real_gdp[:5]

10    $893.31   (2020 est.)$991.52   (2019 est.)$1,0...
15    $463.12   (2020 est.)$495.8   (2019 est.)$488....
16    $138.51   (2020 est.)$144.74   (2019 est.)$141...
20    $793.49   (2020 est.)$775.08   (2019 est.)$716...
23    $557.11   (2020 est.)$594.47   (2019 est.)$584...
Name: Real GDP, dtype: string

In [11]:
real_gdp = pd.DataFrame(real_gdp.str.split(" ", expand = True))
real_gdp.head()
real_gdp.columns

RangeIndex(start=0, stop=13, step=1)

In [12]:
real_gdp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 10 to 246
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       49 non-null     string
 1   1       49 non-null     string
 2   2       49 non-null     string
 3   3       49 non-null     string
 4   4       49 non-null     string
 5   5       49 non-null     string
 6   6       49 non-null     string
 7   7       33 non-null     string
 8   8       33 non-null     string
 9   9       33 non-null     string
 10  10      33 non-null     string
 11  11      32 non-null     string
 12  12      32 non-null     string
dtypes: string(13)
memory usage: 5.4 KB


In [13]:
real_gdp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
10,$893.31,,,(2020,est.)$991.52,,,(2019,"est.)$1,012,670,000,000",(2018,est.),<NA>,<NA>
15,$463.12,,,(2020,est.)$495.8,,,(2019,est.)$488.86,,,(2018,est.)
16,$138.51,,,(2020,est.)$144.74,,,(2019,est.)$141.24,,,(2018,est.)
20,$793.49,,,(2020,est.)$775.08,,,(2019,est.)$716.65,,,(2018,est.)
23,$557.11,,,(2020,est.)$594.47,,,(2019,est.)$584.05,,,(2018,est.)


In [14]:
real_gdp_2020 = real_gdp[0]

In [15]:
real_gdp_2019 = real_gdp[4]

In [16]:
real_gdp_2018 = real_gdp[8]

In [17]:
real_gdp[4]

10                 est.)$991.52
15                  est.)$495.8
16                 est.)$144.74
20                 est.)$775.08
23                 est.)$594.47
25                  est.)$38.79
32      est.)$3,072,550,000,000
36                 est.)$161.78
42                  est.)$94.25
43      est.)$1,808,660,000,000
52                 est.)$734.22
62                  est.)$35.48
64                 est.)$335.36
70      est.)$1,118,720,000,000
76                 est.)$248.97
81      est.)$3,036,490,000,000
88      est.)$4,432,430,000,000
91                 est.)$318.68
108     est.)$8,817,670,000,000
109     est.)$3,043,880,000,000
111                est.)$415.32
112                est.)$433.17
114                est.)$362.23
115     est.)$2,540,890,000,000
122                est.)$101.74
144                est.)$906.24
159                 est.)$279.3
160                 est.)$38.91
165                est.)$982.22
170     est.)$1,009,750,000,000
177     est.)$1,005,850,000,000
183     

In [18]:
len("0 est.)")

7

In [19]:
df["Economy: Real GDP growth rate"]

10     -2.03% (2019 est.)-2.53% (2018 est.)2.83% (201...
15     1.42% (2019 est.)2.58% (2018 est.)2.4% (2017 e...
16     0.1% (2017 est.)-3.1% (2016 est.)0.6% (2015 est.)
20      7.4% (2017 est.)7.2% (2016 est.)6.8% (2015 est.)
23     1.41% (2019 est.)1.49% (2018 est.)1.9% (2017 e...
25        5.6% (2017 est.)4% (2016 est.)2.1% (2015 est.)
32     1.13% (2019 est.)1.2% (2018 est.)1.62% (2017 e...
36     3.39% (2019 est.)3.2% (2018 est.)3.5% (2017 est.)
42      3.5% (2017 est.)4.6% (2016 est.)5.7% (2015 est.)
43     1.66% (2019 est.)2.02% (2018 est.)3.17% (2017 ...
52     3.26% (2019 est.)2.51% (2018 est.)1.36% (2017 ...
62     3.08% (2019 est.)5.25% (2018 est.)5.16% (2017 ...
64     2.85% (2019 est.)2.18% (2018 est.)2.83% (2017 ...
70      4.2% (2017 est.)4.3% (2016 est.)4.4% (2015 est.)
76      10.9% (2017 est.)8% (2016 est.)10.4% (2015 est.)
81     1.49% (2019 est.)1.81% (2018 est.)2.42% (2017 ...
88     0.59% (2019 est.)1.3% (2018 est.)2.91% (2017 e...
91     1.87% (2019 est.)1.91% (

In [20]:
df["Economy: Real GDP per capita"]

10     $19,700 (2020 est.)$22,100 (2019 est.)$22,800 ...
15     $51,900 (2020 est.)$55,800 (2019 est.)$55,300 ...
16     $13,700 (2020 est.)$14,400 (2019 est.)$14,200 ...
20     $4,800 (2020 est.)$4,800 (2019 est.)$4,400 (20...
23     $48,200 (2020 est.)$51,700 (2019 est.)$51,100 ...
25     $3,300 (2020 est.)$3,300 (2019 est.)$3,200 (20...
32     $14,100 (2020 est.)$14,800 (2019 est.)$14,700 ...
36     $22,400 (2020 est.)$23,200 (2019 est.)$22,200 ...
42     $3,600 (2020 est.)$3,600 (2019 est.)$3,600 (20...
43     $45,900 (2020 est.)$49,000 (2019 est.)$48,800 ...
52     $13,400 (2020 est.)$14,600 (2019 est.)$14,300 ...
62     $37,700 (2020 est.)$40,200 (2019 est.)$39,600 ...
64     $55,900 (2020 est.)$57,700 (2019 est.)$56,300 ...
70     $12,000 (2020 est.)$11,800 (2019 est.)$11,400 ...
76     $2,300 (2020 est.)$2,200 (2019 est.)$2,100 (20...
81     $42,000 (2020 est.)$45,800 (2019 est.)$45,300 ...
88     $50,900 (2020 est.)$53,600 (2019 est.)$53,500 ...
91     $27,300 (2020 est.)$29,7

In [21]:
df["Economy: Current account balance"]

10     -$3.997 billion (2019 est.)-$27.049 billion (2...
15     $12.667 billion (2019 est.)$5.989 billion (201...
16     $1.685 billion (2017 est.)-$1.363 billion (201...
20     -$5.322 billion (2017 est.)$1.391 billion (201...
23     $1.843 billion (2019 est.)-$4.135 billion (201...
25     -$1.024 billion (2017 est.)-$808 million (2016...
32     -$50.927 billion (2019 est.)-$41.54 billion (2...
36     $2.06 billion (2019 est.)$611 million (2018 est.)
42     -$932 million (2017 est.)-$1.034 billion (2016...
43     -$35.425 billion (2019 est.)-$42.862 billion (...
52     -$13.748 billion (2019 est.)-$13.118 billion (...
62     -$1.578 billion (2019 est.)-$958 million (2018...
64     $30.935 billion (2019 est.)$24.821 billion (20...
70     -$8.915 billion (2019 est.)-$7.682 billion (20...
76     -$6.551 billion (2017 est.)-$6.574 billion (20...
81     -$18.102 billion (2019 est.)-$16.02 billion (2...
88     $280.238 billion (2019 est.)$297.434 billion (...
91     -$3.114 billion (2019 es

In [22]:
df["Economy: Exports - partners"]

10     Brazil 16%, China 11%, United States 7%, Chile...
15     Germany 28%, United States 7%, Italy 6%, Switz...
16     Italy 28%, Turkey 15%, Israel 7%, Germany 5%, ...
20     United States 15%, Germany 14%, United Kingdom...
23     Germany 17%, France 14%, Netherlands 13%, Unit...
25     Nigeria 25%, Bangladesh 14%, United Arab Emira...
32                   China 28%, United States 13% (2019)
36     Germany 16%, Romania 8%, Italy 7%, Turkey 7%, ...
42     China 17%, Netherlands 14%, Italy 9%, United A...
43                                         US 73% (2019)
52     United States 31%, China 11%, Panama 6%, Ecuad...
62     India 9%, Greece 9%, Libya 8%, United Kingdom ...
64     Germany 14%, United States 11%, Sweden 10%, Un...
70     United States 9%, United Arab Emirates 6%, Ita...
76     China 17%, United States 16%, United Arab Emir...
81     Germany 14%, United States 8%, Italy 7%, Spain...
88     United States 9%, France 8%, China 7%, Netherl...
91     Italy 10%, Germany 7%, T

In [23]:
print(df["Economy: Exports"])

10     $64.18 billion (2020 est.) note: data are in c...
15     $226.79 billion (2020 est.) note: data are in ...
16     $15.21 billion (2020 est.) note: data are in c...
20     $38.78 billion (2020 est.) note: data are in c...
23     $414.79 billion (2020 est.) note: data are in ...
25     $3.58 billion (2019 est.) note: data are in cu...
32     $239.18 billion (2020 est.) note: data are in ...
36     $39.27 billion (2020 est.) note: data are in c...
42     $7.73 billion (2019 est.) note: data are in cu...
43     $477.31 billion (2020 est.) note: data are in ...
52     $39.14 billion (2020 est.) note: data are in c...
62     $16.1 billion (2020 est.) note: data are in cu...
64     $191.53 billion (2020 est.) note: data are in ...
70     $40.1 billion (2020 est.) note: data are in cu...
76     $7.62 billion (2018 est.) note: data are in cu...
81     $746.91 billion (2020 est.) note: data are in ...
88     $1,671,650,000,000 (2020 est.) note: data are ...
91     $59.02 billion (2020 est

In [24]:
df.columns.tolist()

['Country',
 'Economy: Real GDP (purchasing power parity)',
 'Economy: Real GDP growth rate',
 'Economy: Real GDP per capita',
 'Economy: Current account balance',
 'Economy: Exports',
 'Economy: Exports - partners',
 'Economy: Reserves of foreign exchange and gold',
 'Energy: Electricity access - electrification - total population',
 "Economy: Credit ratings - Moody's rating",
 'Economy: Credit ratings - Standard & Poors rating',
 'Economy: Credit ratings - Fitch rating',
 'Economy: Unemployment rate',
 'Economy: Population below poverty line',
 'Economy: Household income or consumption by percentage share - highest 10%',
 'People and Society: Population',
 'People and Society: Nationality - noun',
 'People and Society: Nationality - adjective',
 'People and Society: Languages',
 'People and Society: Religions',
 'People and Society: Age structure - 0-14 years',
 'People and Society: Age structure - 15-24 years',
 'People and Society: Age structure - 25-54 years',
 'People and Society

In [25]:
df['Economy: Reserves of foreign exchange and gold']

10     $55.33 billion (31 December 2017 est.)$38.43 b...
15     $21.57 billion (31 December 2017 est.)$23.36 b...
16     $6.681 billion (31 December 2017 est.)$7.142 b...
20     $33.42 billion (31 December 2017 est.)$32.28 b...
23     $26.16 billion (31 December 2017 est.)$24.1 bi...
25     $698.9 million (31 December 2017 est.)$57.5 mi...
32     $374 billion (31 December 2017 est.)$367.5 bil...
36     $28.38 billion (31 December 2017 est.)$25.13 b...
42     $3.235 billion (31 December 2017 est.)$2.26 bi...
43     $86.68 billion (31 December 2017 est.)$82.72 b...
52     $47.13 billion (31 December 2017 est.)$46.18 b...
62     $888.2 million (31 December 2017 est.)$817.7 m...
64     $75.25 billion (31 December 2017 est.)$64.25 b...
70     $35.89 billion (31 December 2017 est.)$23.2 bi...
76     $3.013 billion (31 December 2017 est.)$3.022 b...
81     $156.4 billion (31 December 2017 est.)$138.2 b...
88     $200.1 billion (31 December 2017 est.)$173.7 b...
91     $7.807 billion (31 Decem

In [26]:
df['Energy: Electricity access - electrification - total population']

10      99% (2020)
15     100% (2020)
16     100% (2020)
20      83% (2019)
23     100% (2020)
25      33% (2019)
32     100% (2020)
36     100% (2020)
42      70% (2019)
43     100% (2020)
52      97% (2019)
62     100% (2020)
64     100% (2020)
70     100% (2020)
76      47% (2019)
81     100% (2020)
88     100% (2020)
91     100% (2020)
108     99% (2019)
109     99% (2019)
111    100% (2020)
112    100% (2020)
114    100% (2020)
115    100% (2020)
122    100% (2020)
144    100% (2020)
159    100% (2020)
160     35% (2019)
165    100% (2020)
170     62% (2019)
177     79% (2019)
183    100% (2020)
184     97% (2019)
185     96% (2019)
187    100% (2020)
188    100% (2020)
191    100% (2020)
192    100% (2020)
215     94% (2019)
217    100% (2020)
219    100% (2019)
223    100% (2020)
224    100% (2020)
235    100% (2020)
236    100% (2020)
240     29% (2019)
243    100% (2020)
244    100% (2020)
246    100% (2020)
Name: Energy: Electricity access - electrification - total population

In [27]:
df["Economy: Unemployment rate"]

10                  9.84% (2019 est.)9.18% (2018 est.)
15                   7.35% (2019 est.)7.7% (2018 est.)
16                        5% (2017 est.)5% (2016 est.)
20                    4.4% (2017 est.)4.4% (2016 est.)
23                  5.36% (2019 est.)5.96% (2018 est.)
25                                      1% (2014 est.)
32                11.93% (2019 est.)12.26% (2018 est.)
36                  5.66% (2019 est.)6.18% (2018 est.)
42                     4.3% (2014 est.)30% (2001 est.)
43                  5.67% (2019 est.)5.83% (2018 est.)
52                  10.5% (2019 est.)9.68% (2018 est.)
62                  7.07% (2019 est.)8.37% (2018 est.)
64                  3.05% (2019 est.)3.07% (2018 est.)
70                  7.86% (2019 est.)12.7% (2016 est.)
76                    17.5% (2012 est.)18% (2011 est.)
81                  8.12% (2019 est.)8.69% (2018 est.)
88                  4.98% (2019 est.)5.19% (2018 est.)
91                 17.3% (2019 est.)19.34% (2018 est.)
108       

In [28]:
df.columns.tolist()

['Country',
 'Economy: Real GDP (purchasing power parity)',
 'Economy: Real GDP growth rate',
 'Economy: Real GDP per capita',
 'Economy: Current account balance',
 'Economy: Exports',
 'Economy: Exports - partners',
 'Economy: Reserves of foreign exchange and gold',
 'Energy: Electricity access - electrification - total population',
 "Economy: Credit ratings - Moody's rating",
 'Economy: Credit ratings - Standard & Poors rating',
 'Economy: Credit ratings - Fitch rating',
 'Economy: Unemployment rate',
 'Economy: Population below poverty line',
 'Economy: Household income or consumption by percentage share - highest 10%',
 'People and Society: Population',
 'People and Society: Nationality - noun',
 'People and Society: Nationality - adjective',
 'People and Society: Languages',
 'People and Society: Religions',
 'People and Society: Age structure - 0-14 years',
 'People and Society: Age structure - 15-24 years',
 'People and Society: Age structure - 25-54 years',
 'People and Society

In [41]:
#df["Economy: Household income or consumption by percentage share - highest 10%"]

In [40]:
#df["People and Society: Population"]

In [37]:
#df["People and Society: Age structure - 0-14 years"]

In [38]:
#df["People and Society: Age structure - 15-24 years"]

In [39]:
#df['People and Society: Age structure - 25-54 years']

In [80]:
##start here

#df["People and Society: Median age - total"] # strip years

In [82]:
#df["People and Society: Median age - male"]

In [54]:
#df["People and Society: Median age - female"] #strip years and estimate date

In [86]:
#df["People and Society: Population growth rate"] #strip estimate date

In [88]:
#df["People and Society: Birth rate"] #strip /1,000 population (2022 est.)

In [90]:
#df["People and Society: Death rate"] #strip /1,000 population (2022 est.)

In [92]:
#df["People and Society: Net migration rate"]

In [94]:
#df["People and Society: Urbanization - urban population"] #strip of total population (2022)

In [96]:
#df["People and Society: Sex ratio - at birth"] #strip male(s)/female

In [101]:
#df["People and Society: Sex ratio - 65 years and over"] #strip male(s)/female

In [102]:
#df['People and Society: Sex ratio - total population'] #strip male(s)/female (2022 est.)

In [104]:
#df["People and Society: Life expectancy at birth - male"] #strip years

In [106]:
#df["People and Society: Life expectancy at birth - female"] # strip years (2022 est.)

In [108]:
#df["People and Society: Total fertility rate"] #strip children born/woman (2022 est.)

In [110]:
#df["People and Society: Obesity - adult prevalence rate"] #strip (2016)

In [112]:
#df["People and Society: Tobacco use - total"] #strip (2020 est.)

In [114]:
#df["Communications: Internet users - total"] # strip (2020 est.)

In [116]:
#df["Communications: Internet users - percent of population"] #strip (2020 est.)

In [76]:
df["Military and Security: Military expenditures"]

10     0.8% of GDP (2021 est.)0.8% of GDP (2020)0.7% ...
15     0.9% of GDP (2021 est.)0.7% of GDP (2020 est.)...
16     5.2% of GDP (2021 est.)5.4% of GDP (2020 est.)...
20     1.2% of GDP (2021 est.)1.3% of GDP (2020 est.)...
23     1.1% of GDP (2021 est.)1% of GDP (2020)0.9% of...
25     0.7% of GDP (2021 est.)0.5% of GDP (2020 est.)...
32     1.3% of GDP (2021 est.)1.4% of GDP (2020)1.4% ...
36     1.6% of GDP (2021 est.)1.6% of GDP (2020)3.1% ...
42     1% of GDP (2021 est.)1% of GDP (2020 est.)1.4%...
43     1.4% of GDP (2021 est.)1.4% of GDP (2020)1.3% ...
52     3% of GDP (2021 est.)3% of GDP (2020 est.)3.1%...
62     2% of GDP (2021 est.)1.8% of GDP (2020 est.)1....
64     1.4% of GDP (2021 est.)1.4% of GDP (2020)1.3% ...
70     1.3% of GDP (2021 est.)1.3% of GDP (2020)1.3% ...
76     0.5% of GDP (2021 est.)0.5% of GDP (2020 est.)...
81     1.9% of GDP (2021 est.)2% of GDP (2020)1.8% of...
88     1.5% of GDP (2021 est.)1.5% of GDP (2020)1.4% ...
91     3.6% of GDP (2021 est.)2

In [77]:
df["Military and Security: Military service age and obligation"]

10     18-24 years of age for voluntary military serv...
15     registration requirement at age 17, the legal ...
16     men age 18-35 years for compulsory military se...
20     16-21 years of age for voluntary military serv...
23     18 years of age for male and female voluntary ...
25     18-35 years of age for selective compulsory an...
32     18-45 years of age for compulsory military ser...
36     18-27 years of age for voluntary military serv...
42     18-23 years of age for male and female volunta...
43     17 years of age for voluntary male and female ...
52     18-24 years of age for compulsory (men) and vo...
62     Cypriot National Guard (CNG): 18-50 years of a...
64     18 years of age for compulsory and voluntary m...
70     voluntary enlistment possible from age 16 (men...
76     18 years of age for voluntary military service...
81     18-25 years of age for male and female volunta...
88     17-23 years of age for male and female volunta...
91     19-45 years of age for c

In [78]:
df["Terrorism: Terrorist group(s)"]

10                                             Hizballah
15             Islamic State of Iraq and ash-Sham (ISIS)
16     Islamic State of Iraq and ash-Sham (ISIS); Isl...
20     Harakat ul-Jihad-i-Islami/Bangladesh; Islamic ...
23     Islamic Revolutionary Guard Corps/Qods Force; ...
25     al-Qa’ida (Jama’at Nusrat al Islam wal Muslime...
32                                      Hizballah (2022)
36     Islamic State of Iraq and ash-Sham (ISIS); Isl...
42     Boko Haram; Islamic State of Iraq and ash-Sham...
43     Islamic State of Iraq and ash-Sham (ISIS); Hiz...
52     National Liberation Army (ELN); Revolutionary ...
62             Islamic State of Iraq and ash-Sham (ISIS)
64     Islamic State of Iraq and ash-Sham (ISIS); Isl...
70     Army of Islam; Islamic State of Iraq and ash-S...
76                           al-Shabaab; IRGC/Qods Force
81     Islamic Revolutionary Guard Corps/Qods Force; ...
88     Islamic Revolutionary Guard Corps/Qods Force; ...
91     Islamic State of Iraq an